In [ ]:
import decimal
import sys

import pandas
import plotly.express

sys.path.append('..')

import src.hashstack
import src.nostra
import src.swap_liquidity
import src.zklend
import update_data

In [ ]:
zklend_events = update_data.get_events(0)
hashstack_events = src.hashstack.get_hashstack_events()
nostra_events = src.nostra.get_nostra_events()

# Iterate over ordered events to obtain the final state of each user.
zklend_state = src.zklend.State()
for _, zklend_event in zklend_events.iterrows():
    zklend_state.process_event(event=zklend_event)
hashstack_state = src.hashstack.State()
for _, hashstack_event in hashstack_events.iterrows():
    hashstack_state.process_event(event=hashstack_event)
nostra_state = src.nostra.State()
for _, nostra_event in nostra_events.iterrows():
    nostra_state.process_event(event=nostra_event)

In [ ]:
prices = src.swap_liquidity.Prices()

swap_amms = await src.swap_liquidity.SwapAmm().init()

In [ ]:
zklend_loan_stats = pandas.DataFrame()
zklend_loan_stats["User"] = [
    user
    for user in zklend_state.user_states.keys()
]
zklend_loan_stats["Protocol"] = "zkLend"
zklend_loan_stats["Borrowing in USD"] = zklend_loan_stats.apply(
    lambda x: src.zklend.compute_borrowings_usd(
        user_state=zklend_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
zklend_loan_stats[
    "Risk adjusted collateral in USD"
] = zklend_loan_stats.apply(
    lambda x: src.zklend.compute_risk_adjusted_collateral_usd(
        user_state=zklend_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
zklend_loan_stats["Health factor"] = zklend_loan_stats.apply(
    lambda x: src.zklend.compute_health_factor(
        risk_adjusted_collateral_usd=x["Risk adjusted collateral in USD"],
        borrowings_usd=x["Borrowing in USD"],
    ),
    axis=1,
)
zklend_loan_stats["Standardized health factor"] = zklend_loan_stats.apply(
    lambda x: src.zklend.compute_standardized_health_factor(
        risk_adjusted_collateral_usd=x["Risk adjusted collateral in USD"],
        borrowings_usd=x["Borrowing in USD"],
    ),
    axis=1,
)

In [ ]:
hashstack_loan_stats = pandas.DataFrame()
hashstack_loan_stats["User"] = [
    user
    for user, user_state in hashstack_state.user_states.items()
    for _ in user_state.loans.keys()
]
hashstack_loan_stats["Protocol"] = "Hashstack"
hashstack_loan_stats["Loan ID"] = [
    loan_id
    for user_state in hashstack_state.user_states.values()
    for loan_id in user_state.loans.keys()
]
hashstack_loan_stats["Borrowing in USD"] = hashstack_loan_stats.apply(
    lambda x: src.hashstack.compute_borrowings_amount_usd(
        borrowings=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .borrowings,
        prices=prices.prices,
    ),
    axis=1,
)
hashstack_loan_stats = hashstack_loan_stats[hashstack_loan_stats['Borrowing in USD'] > decimal.Decimal("0")]
hashstack_loan_stats[
    "Risk adjusted collateral in USD"
] = hashstack_loan_stats.apply(
    lambda x: src.hashstack.compute_collateral_current_amount_usd(
        collateral=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .collateral,
        prices=prices.prices,
    ),
    axis=1,
)
hashstack_loan_stats["Health factor"] = hashstack_loan_stats.apply(
    lambda x: src.hashstack.compute_health_factor(
        borrowings=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .borrowings,
        collateral=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .collateral,
        prices=prices.prices,
        user=x["User"],
    ),
    axis=1,
)
hashstack_loan_stats["Standardized health factor"] = hashstack_loan_stats.apply(
    lambda x: src.hashstack.compute_standardized_health_factor(
        borrowings=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .borrowings,
        collateral=hashstack_state.user_states[x["User"]]
        .loans[x["Loan ID"]]
        .collateral,
        prices=prices.prices,
    ),
    axis=1,
)
hashstack_loan_stats.drop(columns=["Loan ID"], inplace=True)

In [ ]:
nostra_loan_stats = pandas.DataFrame()
nostra_loan_stats["User"] = [
    user
    for user in nostra_state.user_states.keys()
]
nostra_loan_stats["Protocol"] = "Nostra"
nostra_loan_stats["Borrowing in USD"] = nostra_loan_stats.apply(
    lambda x: src.nostra.compute_borrowings_amount_usd(
        user_state=nostra_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
nostra_loan_stats[
    "Risk adjusted collateral in USD"
] = nostra_loan_stats.apply(
    lambda x: src.nostra.compute_risk_adjusted_collateral_usd(
        user_state=nostra_state.user_states[x["User"]],
        prices=prices.prices,
    ),
    axis=1,
)
nostra_loan_stats["Health factor"] = nostra_loan_stats.apply(
    lambda x: src.nostra.compute_health_factor(
        risk_adjusted_collateral_usd=x["Risk adjusted collateral in USD"],
        risk_adjusted_debt_usd=x["Borrowing in USD"],
    ),
    axis=1,
)
nostra_loan_stats["Standardized health factor"] = nostra_loan_stats.apply(
    lambda x: src.nostra.compute_standardized_health_factor(
        risk_adjusted_collateral_usd=x["Risk adjusted collateral in USD"],
        borrowings_usd=x["Borrowing in USD"],
    ),
    axis=1,
)

In [ ]:
standardized_health_factors = pandas.DataFrame()
standardized_health_factors['zkLend'] = zklend_loan_stats['Standardized health factor']
standardized_health_factors['Hashstack'] = hashstack_loan_stats['Standardized health factor']
standardized_health_factors['Nostra'] = nostra_loan_stats['Standardized health factor']

standardized_health_factors.where(standardized_health_factors > decimal.Decimal('0'), decimal.Decimal('NaN'), inplace=True)
standardized_health_factors.where(standardized_health_factors < decimal.Decimal('5'), decimal.Decimal('NaN'), inplace=True)
standardized_health_factors

In [ ]:
figure = plotly.express.histogram(
    data_frame=standardized_health_factors.astype(float),
    x=[
        "zkLend",
        "Hashstack",
        "Nostra",
    ],
    nbins=100,
    title=f"Standardized health factor",
    barmode="overlay",
    opacity=0.65,
    color_discrete_map={
        "zkLend": '#ECD662',
        "Hashstack": '#4CA7D0',
        "Nostra": '#8FCE00',
    },
)
figure.show()